In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.ensemble import GradientBoostingRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.preprocessing import RobustScaler
from scipy.stats import boxcox_normmax, zscore
from multiprocessing import cpu_count
from lightgbm import LGBMRegressor
from scipy.special import boxcox1p
import matplotlib.pyplot as plt
from sklearn.svm import SVR
import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')

In [ ]:
df['SalePrice'].describe()

In [ ]:
var = 'GrLivArea'
data = pd.concat([df['SalePrice'], df[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

In [ ]:
var = 'TotalBsmtSF'
data = pd.concat([df['SalePrice'], df[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

In [ ]:
var = 'OverallQual'
data = pd.concat([df['SalePrice'], df[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);

In [ ]:
var = 'YearBuilt'
data = pd.concat([df['SalePrice'], df[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);

In [ ]:
corrmat = df.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True)

In [ ]:
var = 'GarageArea'
data = pd.concat([df['SalePrice'], df[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df[cols], size = 2.5)
plt.show();

In [ ]:
train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv", index_col=0)
test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv", index_col=0)
sample = pd.read_csv("../input/house-prices-advanced-regression-techniques/sample_submission.csv")

In [ ]:
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.grid()
ax.scatter(train["YrSold"], train["SalePrice"], c="#3f72af", zorder=1, alpha=0.9)
ax.axvline(4500, c="#112d4e", ls="--", zorder=2)
ax.set_xlabel("Ground living area (sq. ft)", labelpad=10)
ax.set_ylabel("Sale price ($)", labelpad=10)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.grid()
ax.scatter(train["GrLivArea"], train["SalePrice"], c="#3f72af", zorder=3, alpha=0.9)
ax.axvline(4500, c="#112d4e", ls="--", zorder=2)
ax.set_xlabel("Ground living area (sq. ft)", labelpad=10)
ax.set_ylabel("Sale price ($)", labelpad=10)
plt.show()

In [ ]:
numVal = pd.get_dummies(train)
num = 0
for elem in numVal.columns:
    print(num, elem)
    num += 1

In [ ]:
num  = 0
for elem in numVal.columns:
    num += 1
    print(num)
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.grid()
    ax.scatter(train[elem], train["SalePrice"], c="#3f72af", zorder=3, alpha=0.9)
    ax.axvline(4500, c="#112d4e", ls="--", zorder=2)
    ax.set_xlabel(elem, labelpad=10)
    ax.set_ylabel("Sale price ($)", labelpad=10)
    plt.show()
    if num == 37:
        break;
    

In [ ]:
#LotArea, YearBuilt, MasVnrArea, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, 1stFlrSF, 2ndFlrSF, GrLivArea, GarageArea,

In [ ]:
lotarea = 35000
bsmtfinsf1 = 2200
totalbsmtsf = 3500
firstflrsf = 3200
grlivarea = 3800
garagearea = 1200

train["GrLivArea"] = np.where(train["GrLivArea"] > grlivarea, grlivarea, train['GrLivArea'])
train["LotArea"] = np.where(train["LotArea"] > lotarea, lotarea, train['LotArea'])
train["BsmtFinSF1"] = np.where(train["BsmtFinSF1"] > bsmtfinsf1, bsmtfinsf1, train['BsmtFinSF1'])
train["TotalBsmtSF"] = np.where(train["TotalBsmtSF"] > totalbsmtsf, totalbsmtsf, train['TotalBsmtSF'])
train["1stFlrSF"] = np.where(train["1stFlrSF"] > firstflrsf, firstflrsf, train['1stFlrSF'])
train["GarageArea"] = np.where(train["GarageArea"] > firstflrsf, firstflrsf, train['GarageArea'])

In [ ]:
num  = 0
for elem in numVal.columns:
    num += 1
    print(num)
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.grid()
    ax.scatter(train[elem], train["SalePrice"], c="#3f72af", zorder=3, alpha=0.9)
    ax.axvline(4500, c="#112d4e", ls="--", zorder=2)
    ax.set_xlabel(elem, labelpad=10)
    ax.set_ylabel("Sale price ($)", labelpad=10)
    plt.show()
    if (num == 37):
        break


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.grid()
ax.scatter(train['GarageArea'], train["SalePrice"], c="#3f72af", zorder=3, alpha=0.9)
ax.axvline(4500, c="#112d4e", ls="--", zorder=2)
ax.set_xlabel("garage area", labelpad=10)
ax.set_ylabel("Sale price ($)", labelpad=10)
plt.show()

In [ ]:
#train = train[train["GrLivArea"] < 4500]
#train = train[train["LotArea"]< 100000]
#train = train[train["BsmtFinSF1"]< 3000]
#train = train[train["TotalBsmtSF"]< 4000]
#train = train[train["1stFlrSF"]< 4000]
#train = train[train["2ndFlrSF"]< 4000]

In [ ]:
y_train = train["SalePrice"]

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(y_train, label=f'train target, skew: {y_train.skew():.2f}')
plt.legend(loc='best')
plt.show()

In [ ]:
y_train = np.log(train["SalePrice"])

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(y_train, label=f'train target, skew: {y_train.skew():.2f}')
plt.legend(loc='best')
plt.show()

In [ ]:
train.shape

In [ ]:
temp = pd.concat([train.drop("SalePrice", axis=1), test])

In [ ]:
tempcol =["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu", "GarageCond", "GarageQual", "GarageFinish", "GarageType", "BsmtCond", "BsmtExposure", "BsmtQual", "BsmtFinType2", "BsmtFinType1"]
temp[tempcol] = temp[tempcol].fillna("None")
tempcol = ["GarageYrBlt", "MasVnrArea", "BsmtHalfBath", "BsmtFullBath", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "GarageCars"]
temp[tempcol] = temp[tempcol].fillna(0)
tempcol = ["MasVnrType", "MSZoning", "Utilities", "Exterior1st", "Exterior2nd", "SaleType", "Electrical", "KitchenQual", "Functional"]
temp[tempcol] = temp.groupby("Neighborhood")[tempcol].transform(lambda x: x.fillna(x.mode()[0]))
tempcol = ["GarageArea", "LotFrontage"]
temp[tempcol] = temp.groupby("Neighborhood")[tempcol].transform(lambda x: x.fillna(x.median()))

In [ ]:
temp["TotalSF"] = temp["GrLivArea"] + temp["TotalBsmtSF"]
temp["TotalPorchSF"] = temp["OpenPorchSF"] + temp["EnclosedPorch"] + temp["3SsnPorch"] + temp["ScreenPorch"]
temp["TotalBath"] = temp["FullBath"] + temp["BsmtFullBath"] + 0.5 * (temp["BsmtHalfBath"] + temp["HalfBath"])

In [ ]:
col = ["MSSubClass", "YrSold"]
temp[col] = temp[col].astype("category")

In [ ]:
skew = temp.skew(numeric_only=True).abs()
col = skew[skew > 1].index
for c in col:
    temp[c] = boxcox1p(temp[c], boxcox_normmax(temp[c] + 1))

In [ ]:
temp

In [ ]:
col = temp.select_dtypes(np.number).columns
temp[col] = RobustScaler().fit_transform(temp[col])

In [ ]:
temp = pd.get_dummies(temp)

In [ ]:
X_train = temp.loc[train.index]
X_test = temp.loc[test.index]

In [ ]:
residuals = y_train - LinearRegression().fit(X_train, y_train).predict(X_train)
outliers = residuals[np.abs(zscore(residuals)) > 3].index

In [ ]:
X_train = X_train.drop(outliers)
y_train = y_train.drop(outliers)

In [ ]:
kf = KFold(n_splits=5, random_state=0, shuffle=True)
rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))
scorer = make_scorer(rmse, greater_is_better=False)
n_jobs = max(cpu_count() - 2, 1)
n_iter = 100

In [ ]:
ridgeSearch =  RandomizedSearchCV( Ridge(), {"alpha": np.logspace(-1, 2, 500)}, n_iter, scorer, cv=kf, random_state=0, verbose=True)
LassoSearch = RandomizedSearchCV(Lasso(), {"alpha": np.logspace(-5, -1, 500)}, n_iter, scorer, n_jobs=n_jobs, cv=kf, random_state=0, verbose=True)
svrSearch = RandomizedSearchCV(SVR(), {"C": np.arange(1, 100), "gamma": np.linspace(0.00001, 0.001, 50), "epsilon": np.linspace(0.01, 0.1, 50)}, n_iter, scorer, n_jobs=n_jobs, cv=kf, random_state=0, verbose=True)
lgbmSearch = RandomizedSearchCV(LGBMRegressor(n_estimators=2000, max_depth=3), {"colsample_bytree": np.linspace(0.2, 0.7, 6), "learning_rate": np.logspace(-3, -1, 100)}, n_iter, scorer, n_jobs=n_jobs, cv=kf, random_state=0, verbose=True)
gbmSearch = RandomizedSearchCV(GradientBoostingRegressor(n_estimators=2000, max_depth=3), {"max_features": np.linspace(0.2, 0.7, 6), "learning_rate": np.logspace(-3, -1, 100)}, n_iter, scorer, n_jobs=n_jobs, cv=kf, random_state=0, verbose=True)

In [ ]:
ridgeSearch= ridgeSearch.fit(X_train, y_train)
lassoSearch= LassoSearch.fit(X_train, y_train)
svrSearch= svrSearch.fit(X_train, y_train)
lgbmSearch= lgbmSearch.fit(X_train, y_train)
gbmSearch= gbmSearch.fit(X_train, y_train)

In [ ]:
models = [search.best_estimator_ for search in [ridgeSearch, lassoSearch, svrSearch, lgbmSearch, gbmSearch]]
stack_search = random_search(StackingCVRegressor(models, Lasso(), cv=kf), {"meta_regressor__alpha": np.logspace(-3, -2, 500)}, n_iter=20)
models.append(stack_search.best_estimator_)

In [ ]:
preds = [model.predict(X_test) for model in models]

In [ ]:
preds2 = np.average(preds, axis=0, weights=None)

In [ ]:
preds2 = np.exp(preds)

In [ ]:
preds2

In [ ]:
idhold = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
overall = (preds2[0]  + preds2[1]  + preds2[2]   + preds2[3]  + preds2[4])/5 

In [ ]:
overall

In [ ]:
comb = pd.DataFrame({"id":idhold.Id, "SalePrice":overall})
comb.to_csv("predictioncomb.csv", index = False)

In [ ]:
dtrain = xgb.DMatrix(X_train, label = y_train)
dtest = xgb.DMatrix(X_test)

params = {"max_depth":2, "eta":0.1}
modelxgb = xgb.cv(params, dtrain,  num_boost_round=500, early_stopping_rounds=100)

In [ ]:
modelxgb.loc[30:,["test-rmse-mean", "train-rmse-mean"]].plot()

In [ ]:
model_xgb = xgb.XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1) 
model_xgb.fit(X_train, y_train)

In [ ]:
xgb_preds = np.expm1(model_xgb.predict(X_test))

In [ ]:
xgb_preds